In [ ]:
###ORLANDO ALFONSO BENAVIDES PEREZ

In [1]:
#Se importan las librerias
import pandas as pd
import math
import json
import numpy as np
import random
from datetime import timedelta

import sagemaker
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.inputs import TrainingInput
import boto3

In [25]:
#Se inicializan los bucket
np.random.seed(42)
random.seed(42)

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
role='LabRole'
s3_bucket = 'dollar-predictor274'
s3_data_path = "s3://{}/{}/data".format(s3_bucket, 'deepar-model')
s3_output_path = "s3://{}/{}/output".format(s3_bucket, 'deepar-model')

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [6]:
# Se carga el dataset
df = pd.read_csv('df_dollar_10min.csv', parse_dates=['fecha'])

In [7]:
# Se divididen los datos  del dataset en datos de entrenamiento y datos de test
train_size = math.ceil(len(df) * 0.7)
train_data = df[:train_size]
test_data = df[train_size:]

In [95]:
freq = '10min' # La serie tiene registros cada 10 min
prediction_length = 1 # se predice para el siguiete periodo
context_length = 12 # 

In [96]:
start_dataset = pd.Timestamp("2023-02-17 13:15:00", freq=freq)
end_training = pd.Timestamp("2023-03-22 17:30:00", freq=freq)

/tmp/ipykernel_18342/1049242397.py:1: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start_dataset = pd.Timestamp("2023-02-17 13:15:00", freq=freq)
/tmp/ipykernel_18342/1049242397.py:2: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  end_training = pd.Timestamp("2023-03-22 17:30:00", freq=freq)


In [97]:
df_daily_groups_train_data = train_data.groupby(pd.Grouper(key='fecha', freq='D'))
df_daily_groups_test_data = test_data.groupby(pd.Grouper(key='fecha', freq='D'))

In [98]:
training_data = [
    {
        "start": str(group.iloc[0]['fecha']),
        "target": group['precio'].values.tolist(),
    }
    for name, group in df_daily_groups_train_data if len(group) > 0
]
print(training_data[:5])

[{'start': '2023-02-17 13:10:00', 'target': [4946.8202, 4942.746043137256, 4937.931349152543, 4933.719009876543, 4931.0680764705885, 4931.236175, 4931.441344186046, 4932.41604, 4934.126216541354, 4934.819030555555, 4935.328091489361, 4934.921802380953, 4933.430054347826, 4931.38086, 4930.012629166667, 4928.255918181818, 4926.226331372549, 4924.88836, 4923.76967027027, 4922.9587833333335, 4922.482694444445, 4922.013855, 4921.494372222222, 4921.098806060606, 4920.75665]}, {'start': '2023-02-20 13:10:00', 'target': [4890.10752, 4892.943850000001, 4894.356809999999, 4898.778865517242, 4904.442151162791, 4908.378322222223, 4909.985377142857, 4911.573092307693, 4912.019033333333, 4912.238271428571, 4912.2645, 4912.2906, 4912.328875, 4912.37695, 4913.6919, 4913.73885, 4913.7861, 4913.925657142858, 4913.982333333333, 4914.142630769231, 4914.341209090909, 4914.4217, 4914.4267, 4914.4365, 4914.4387]}, {'start': '2023-02-21 13:10:00', 'target': [4919.366678571429, 4918.425358208955, 4922.93408913

In [99]:
testing_data = [
    {
        "start": str(group.iloc[0]['fecha']),
        "target": group['precio'].values.tolist(),
    }
    for name, group in df_daily_groups_test_data if len(group) > 0
]
print(testing_data[5:])

[{'start': '2023-03-22 13:00:00', 'target': [4797.672655, 4796.560152173913, 4787.514345454546, 4784.140872093023, 4783.463295454546, 4781.217283333333, 4779.098413636363, 4777.875138888889, 4776.260331111111, 4774.896042, 4774.018496774193, 4773.509864705883, 4773.56224, 4774.0225, 4774.0232166666665, 4773.910484615385, 4773.944192857142, 4774.167964285714, 4774.378694871795, 4774.806668888888, 4775.166169230769, 4775.359740625, 4775.570693181819, 4775.654693333333, 4775.740647826087, 4775.853919148936, 4775.998655, 4776.132548, 4776.124059090909, 4776.21652972973, 4776.2492]}, {'start': '2023-03-23 13:00:00', 'target': [4764.457161290322, 4763.255968115942, 4761.08185730337, 4758.80404494382, 4757.429940983607, 4756.668765306123, 4756.584083333333, 4757.036705555556, 4757.1301875, 4757.3571125, 4757.8938777777785, 4758.14598490566, 4758.319503125, 4758.552785, 4758.865178947369, 4758.891453571428, 4758.531924, 4758.446760606061, 4758.310716666667, 4757.559637864078, 4756.936239215686

In [100]:
# Guardo el archivo json para ser usado luego en el entranamiento
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [101]:
%%time
write_dicts_to_file("train.json", training_data)
write_dicts_to_file("test.json", testing_data)

CPU times: user 0 ns, sys: 2.54 ms, total: 2.54 ms
Wall time: 2.89 ms


In [102]:
s3 = boto3.resource("s3")

def copy_to_s3(local_file, s3_path, override=True):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(
                "File s3://{}/{} already exists.\nSet override to upload anyway.\n".format(
                    s3_bucket, s3_path
                )
            )
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print("Uploading file to {}".format(s3_path))
        buk.put_object(Key=path, Body=data)

In [103]:
%%time
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

Overwriting existing file
Uploading file to s3://dollar-predictor274/deepar-model/data/train/train.json
Overwriting existing file
Uploading file to s3://dollar-predictor274/deepar-model/data/test/test.json
CPU times: user 60 ms, sys: 2.17 ms, total: 62.2 ms
Wall time: 341 ms


In [104]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    base_job_name="dollar-predictor",
    output_path=s3_output_path,
)

In [105]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [62]:
estimator.set_hyperparameters(**hyperparameters)

In [63]:
%%time
data_channels = {"train": "{}/train/".format(s3_data_path), "test": "{}/test/".format(s3_data_path)}

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [64]:
estimator.fit(inputs=data_channels, wait=True)

INFO:sagemaker:Creating training-job with name: dollar-predictor-2023-04-10-20-31-00-048


2023-04-10 20:31:00 Starting - Starting the training job...
2023-04-10 20:31:18 Starting - Preparing the instances for training......
2023-04-10 20:32:22 Downloading - Downloading input data
2023-04-10 20:32:22 Training - Downloading the training image........Arguments: train
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:78: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
/opt/amazon/lib/python3.8/site-packages/jsonref.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, MutableMapping, Sequence
[04/10/2023 20:33:44 INFO 139919854057280] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'_kvstore': 'auto', '_num_gpus': 'auto', '_num_kv_servers': 'auto', '_tuning_objective_metric': '', 'cardinality': 'auto',

In [70]:
from sagemaker.serializers import IdentitySerializer

In [71]:
# Esta clase permiti recibir 
class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            # serializer=JSONSerializer(),
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def predict(
        self,
        ts,
        cat=None,
        dynamic_feat=None,
        num_samples=100,
        return_samples=False,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(
            ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None
        )

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles,
        }

        http_request_data = {"instances": [instance], "configuration": configuration}

        return json.dumps(http_request_data).encode("utf-8")

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode("utf-8"))["predictions"][0]
        prediction_length = len(next(iter(predictions["quantiles"].values())))
        prediction_index = pd.date_range(
            start=prediction_time, freq=freq, periods=prediction_length
        )
        if return_samples:
            dict_of_samples = {"sample_" + str(i): s for i, s in enumerate(predictions["samples"])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(
            data={**predictions["quantiles"], **dict_of_samples}, index=prediction_index
        )

    def set_frequency(self, freq):
        self.freq = freq


def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]


def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

In [73]:
# 2) Desplegar en Sagemaker un modelo que prediga el precio del dólar(
predictor = estimator.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", predictor_cls=DeepARPredictor
)

INFO:sagemaker:Creating model with name: dollar-predictor-2023-04-10-20-35-49-202
INFO:sagemaker:Creating endpoint-config with name dollar-predictor-2023-04-10-20-35-49-202
INFO:sagemaker:Creating endpoint with name dollar-predictor-2023-04-10-20-35-49-202


--------!